In [1]:
import os
import numpy as np
import h5py
from tqdm import tqdm_notebook
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.models import Model

/global/homes/k/ktran/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
# Download ResNet
base_model = ResNet50(weights='imagenet')

# Take a look at what layers there are
for layer in base_model.layers:
    print('%s has output shape of %s' % (layer.name, layer.output_shape))

# Create a function to pull out the features
model = Model(inputs=base_model.input,
              outputs=base_model.get_layer('avg_pool').output)

input_2 has output shape of (None, 224, 224, 3)
conv1_pad has output shape of (None, 230, 230, 3)
conv1 has output shape of (None, 112, 112, 64)
bn_conv1 has output shape of (None, 112, 112, 64)
activation_50 has output shape of (None, 112, 112, 64)
pool1_pad has output shape of (None, 114, 114, 64)
max_pooling2d_2 has output shape of (None, 56, 56, 64)
res2a_branch2a has output shape of (None, 56, 56, 64)
bn2a_branch2a has output shape of (None, 56, 56, 64)
activation_51 has output shape of (None, 56, 56, 64)
res2a_branch2b has output shape of (None, 56, 56, 64)
bn2a_branch2b has output shape of (None, 56, 56, 64)
activation_52 has output shape of (None, 56, 56, 64)
res2a_branch2c has output shape of (None, 56, 56, 256)
res2a_branch1 has output shape of (None, 56, 56, 256)
bn2a_branch2c has output shape of (None, 56, 56, 256)
bn2a_branch1 has output shape of (None, 56, 56, 256)
add_17 has output shape of (None, 56, 56, 256)
activation_53 has output shape of (None, 56, 56, 256)
res2b_b

In [20]:
# Enumerate all of the WGA images
img_folder = './images/'
img_names = [img_name for img_name in os.listdir(img_folder)]
img_names.remove('.gitignore')
import random; img_names = random.sample(img_names, 1000)  # For quick testing
img_paths = [img_folder + img_name for img_name in img_names]

# Pull out the ResNet features for each image
features = np.empty((0, model.output_shape[1]))
for img_path in tqdm_notebook(img_paths):
    img = image.load_img(img_path, target_size=(224, 224))
    array = image.img_to_array(img)
    array_expanded = np.expand_dims(array, axis=0)
    preprocessed_input = preprocess_input(array_expanded)
    features_ = model.predict(preprocessed_input)
    features = np.append(features, features_, axis=0)

In [23]:
# Save features
h5f = h5py.File('features.h5', 'w')
h5f.create_dataset('ResNet_features', data=features)
h5f.close()

# Save the keys
with open('image_mapping.txt', 'w') as file_handle:
    for img_name in img_names:
        file_handle.write("%s\n" % img_name)